In [1]:
import pandas as pd
import numpy as np
import pickle
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from ml_features.features import create_features
from ml_simulation.dataset_split import customer_split
from ml_simulation.util import HiddenPrints
from ml_features.features import create_features
from ml_training.train_xgb import train_xgb
from ml_inference.inference import safe_predict
from ml_simulation__follow_up.data import get_simulation_compute_function
from ml_simulation__follow_up.widget import show_follow_up_widget
    

import warnings
warnings.filterwarnings('ignore')

# Load data
df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])

In [2]:
split_result = customer_split(df_quotes)
df_train = split_result['train']
df_sim = split_result['simulation']




SPLIT CUSTOMERS: TRAIN vs SIMULATION: TRAINING SIZE 0.95
Split: 22708 train, 1180 sim customers


In [3]:
TRAIN = False

In [4]:
if TRAIN:
    with HiddenPrints():
        X_train = create_features(df_train)
    
    y_train = X_train['converted']
    X_train = X_train.drop(['numero_compte', 'converted'], axis=1)
    feature_names = X_train.columns.tolist()
    
    result = train_xgb(X_train, y_train, "simulation_poc")
    model = result['model']
    feature_names = result['features']
    
    print(f"Model trained: {len(feature_names)} features")
else:
    with open('simulation_poc.pkl', 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        feature_names = model_data['features']

In [5]:
import time
sim_conv = df_sim.groupby('numero_compte')['fg_devis_accepte'].max()
non_conv_customers = sim_conv[sim_conv == 0].index.tolist()
print(f"Non-converted customers: {len(non_conv_customers)}")

# At the top of the notebook / in a separate cell
# SAMPLE_SEED = 4474          # ← you can change or comment this
SAMPLE_SEED = int(time.time() * 1000) % 10000000

print(f"Using seed: {SAMPLE_SEED}")
random.seed(SAMPLE_SEED)
# ... your sampling code ..

# Step 1: Get all single-quote customers in one pass (no predictions yet)
single_quote_raw = []
for cust in non_conv_customers:
    quotes = df_sim[df_sim['numero_compte'] == cust]
    if len(quotes) == 1:
        single_quote_raw.append({
            'customer_id': cust,
            'product': quotes['famille_equipement_produit'].iloc[0],
            'price': quotes['mt_apres_remise_ht_devis'].sum(),
            'quotes': quotes
        })

print(f"✅ Found {len(single_quote_raw)} customers with exactly 1 quote")

# Step 2: Sample diverse products FIRST (without predictions)
product_groups = {}
for cust in single_quote_raw:
    prod = cust['product']
    if prod not in product_groups:
        product_groups[prod] = []
    product_groups[prod].append(cust)

selected_raw = []
products_used = set()

for prod, cust_list in product_groups.items():
    if len(selected_raw) >= 5:
        break
    if prod not in products_used and cust_list:
        selected_raw.append(random.choice(cust_list))
        products_used.add(prod)

if len(selected_raw) < 5:
    remaining = [c for c in single_quote_raw if c not in selected_raw]
    selected_raw.extend(random.sample(remaining, 5 - len(selected_raw)))

print(selected_raw)

Non-converted customers: 923
Using seed: 2719706
✅ Found 691 customers with exactly 1 quote
[{'customer_id': 'CL00339749', 'product': 'Pompe à chaleur', 'price': np.float64(13013.14), 'quotes':                 id_devis   num_devis                     nom_devis nom_agence  \
4938  0Q0To000001avFJKAY  DV00183116  DV00183116 - Quentin CHASSET     Aujard   

     nom_filiale_zone     nom_region                 statut_devis  \
4938           Aujard  Île-de-France  En attente de retour client   

      fg_devis_emis  fg_devis_refuse  fg_devis_accepte  ...  \
4938            1.0              0.0               0.0  ...   

     fonction_commercial regroup_famille_equipement_produit  marque_produit  \
4938          Commercial                          HEAT_PUMP        ATLANTIC   

                                         modele_produit  dt_facture_min  \
4938  PAC Air/Eau ALFEA EXCELLIA S DUO 14 kW - monop...             NaN   

      dt_facture_max  dt_prem_contrat  dt_prem_devis  statut_client

In [6]:
# Step 3: Compute predictions ONLY for selected 5 customers
selected = []
for cust in selected_raw:
    baseline = safe_predict(cust['customer_id'], cust['quotes'], model, feature_names)
    selected.append({
        'customer_id': cust['customer_id'],
        'product': cust['product'],
        'price': cust['price'],

        'baseline': baseline,
        'quote_count': 1
    })

import numpy as np
import pandas as pd
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

print("\n🎯 SELECTED 5 SINGLE-QUOTE CUSTOMERS:")
for i, cust in enumerate(selected):
    print(f" {i+1}. {cust['customer_id']} - {cust['product']} (baseline: {cust['baseline']:.3f})")

# Make sure we have consistent lists / dicts
selected_ids = [c['customer_id'] for c in selected]


🎯 SELECTED 5 SINGLE-QUOTE CUSTOMERS:
 1. CL00339749 - Pompe à chaleur (baseline: 0.729)
 2. CL00063542 - Photovoltaïque (baseline: 0.279)
 3. CL00340631 - Chaudière (baseline: 0.532)
 4. CL00345508 - Poêle (baseline: 0.341)
 5. CL00339702 - Autres (baseline: 0.589)


In [7]:
# Simulation
compute = get_simulation_compute_function(model, feature_names, df_sim, selected_ids)
show_follow_up_widget(compute, selected_ids)


=== CREATING BASELINE PREDICTIONS ===
